In [ ]:
import cv2
import numpy as np
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt

# paths to train and test datasets
train_data_dir = '/train'
test_data_dir = '/test'

# load and preprocess images from a folder
def load_and_preprocess_data(folder_path):
    images = []
    labels = []

    # mapping from class labels to integer indices
    class_mapping = {label: idx for idx, label in enumerate(os.listdir(folder_path))}
    
    # Iterate through subfolders
    for label in os.listdir(folder_path):
        label_folder = os.path.join(folder_path, label)
        
        # Getting the integer label from the mapping
        int_label = class_mapping[label]
        
        # Iterate through images in the subfolder
        for image_name in os.listdir(label_folder):
            image_path = os.path.join(label_folder, image_name)
            
            # Read and resize the image
            image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            image = cv2.resize(image, (48, 48))
            
            # Normalize the pixel values
            image = image.astype('float32') / 255.0
            
            # channel dimension to the image
            image = np.expand_dims(image, axis=-1)
            
            images.append(image)
            labels.append(int_label)

    images = np.array(images)
    labels = to_categorical(labels, num_classes=len(class_mapping))

    return images, labels

# Load and preprocess training data
train_images, train_labels = load_and_preprocess_data(train_data_dir)

# Load and preprocess testing data
test_images, test_labels = load_and_preprocess_data(test_data_dir)

# CNN model
def build_model(input_shape, num_classes):
    model = Sequential()

    model.add(Conv2D(64, (3, 3), activation='relu', input_shape=input_shape))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(optimizer=Adam(learning_rate=0.0001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model

# Training the model
input_shape = train_images.shape[1:]
num_classes = train_labels.shape[1]
model = build_model(input_shape, num_classes)

batch_size = 32
epochs = 20

history = model.fit(train_images, train_labels, 
                    epochs=epochs, 
                    batch_size=batch_size, 
                    validation_split=0.1)

# Evaluating the model on the test set
test_loss, test_accuracy = model.evaluate(test_images, test_labels, verbose=2)
print(f'Test Accuracy: {test_accuracy*100:.2f}%')

# Saving the trained model
model.save("facial_emotion_model.h5")

# Plot training history
plt.figure(figsize=(12, 4))

# Plot training & validation accuracy values
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='upper left')

# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.tight_layout()
plt.show()